In [1]:
import pandas as pd

df = pd.read_csv("puf_imputed_cps_person.csv.gz", compression="gzip")

In [2]:
df.partnership_s_corp_income.describe()

count    1.527320e+05
mean     1.028752e+05
std      1.383520e+06
min     -4.711000e+07
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      8.713000e+07
Name: partnership_s_corp_income, dtype: float64

In [3]:
import plotly.express as px

px.imshow(
    df.dropna().corr(),
)

In [4]:
df.isna().any()

person_id                     False
tax_unit_id                   False
employment_income             False
self_employment_income        False
partnership_s_corp_income     False
farm_income                   False
farm_rent_income              False
short_term_capital_gains      False
long_term_capital_gains       False
tax_exempt_interest_income    False
rental_income                 False
qualified_dividend_income     False
taxable_pension_income        False
social_security               False
dtype: bool